In [ ]:
from shapely.geometry import Point, Polygon
import geopandas as gpd
import pandas as pd
import folium
import geojson
import uuid

In [ ]:
posters = pd.read_csv('./dummy/p.csv')

posters['id'] = posters['id'].apply(lambda x : 'P' + str(x))

posters = posters.set_index('id')

geometry = [Point(lon,lat) for lon, lat in zip(posters['longitude'], posters['latitude'] )]

posters_gdf = gpd.GeoDataFrame(posters, geometry=geometry,crs='EPSG:4326' )

posters_gdf = posters_gdf.to_crs("epsg:31370")

posters_gdf.head()



In [ ]:
with open('./dummy/test_geo.geojson', 'r') as f:
    gj = geojson.load(f)

gj.is_valid

gj


In [ ]:
poi = gpd.read_file("./dummy/test_geo.geojson",crs='EPSG:4326').set_index('name')
poi = poi.to_crs("epsg:31370")

In [ ]:
dt = Polygon(gj['features'][0]['geometry']['coordinates'][0])
dt_lon, dt_lat = gj['features'][0]['properties']['center']
map = folium.Map(location=(dt_lat,dt_lon), tiles="cartodbpositron", min_zoom=11)

folium.Marker((dt_lat,dt_lon)).add_to(map)

for index,row in posters_gdf.iterrows():
    lon = row['longitude']
    lat = row['latitude']
    point = Point(lon,lat)

    if point.within(dt) or point.touches(dt):
        folium.Marker(location= [lat, lon],popup=index, icon= folium.Icon(color='red', icon='star') ).add_to(map)
        folium.Circle([lat, lon],radius=300, color='red', fill_color = 'red').add_to(map)

    else:
        folium.Marker(location= [lat, lon],popup=index, icon= folium.Icon(color='cadetblue', icon='star') ).add_to(map)


folium.features.GeoJson(gj).add_to(map)

map

In [ ]:
distances = posters_gdf.geometry.distance(poi.loc['Meir'].geometry.centroid)

distances

In [ ]:
buffer = posters_gdf.geometry.buffer(300)
print(type(buffer))
buffer.crs

posters_gdf.head(50)


In [ ]:
buffer_union = buffer.geometry.unary_union
buffer_union

In [ ]:
buffer = gpd.GeoSeries(buffer_union, crs="epsg:31370")
buffer = buffer.explode(index_parts=True)

uuid_list = [uuid.uuid4() for _ in range(len(buffer))]

buffer.index = uuid_list

buffer.head()

# folium.features.GeoJson(buffer.to_crs(epsg=4324)).add_to(map)

# map

In [ ]:
# for index, row in buffer.items():
#     print(index[1])

def get_buffer_id(point):
    for index, row in buffer.items():
        if point.within(row):
            return index

posters_gdf['buffer_id'] = posters_gdf['geometry'].apply(lambda x : get_buffer_id(x))

posters_gdf

In [ ]:
buffer.loc[(0,3)].centroid

In [ ]:
# for each buffer_id
# get a sub posters_gdf containing only the rows that contain same buffer_id
# calculate distance to centroid, and eliminate panel from group
# do this until buffers contain only one point

for index, row in buffer.items():
    sub_gdf = posters_gdf.loc[posters_gdf['buffer_id'] == index]
    if len(sub_gdf) > 1:
        dist_to_centroid = sub_gdf.geometry.distance(row.centroid)
        print(dist_to_centroid.idxmin())